# Tasca M10 T01 - Web scraping i automatització

En este ejercicio utilizaremos tres librerías, BeautifulSoup, Selenium y Scrapy para hacer scraping de tres sitios diferentes. 

En el primer ejercicio haremos scraping con BeautifulSoup y Selenium en 'Quotes to scrape' (https://quotes.toscrape.com/)

En el segundo ejercicio haremos scraping con BeautifulSoup y Selenium en el sitio de Wikipedia '2022–23 New York Knicks season' (https://en.wikipedia.org/wiki/2022%E2%80%9323_New_York_Knicks_season)

En el tercer ejercicio documentaremos el dataset del ejercicio 2 y produciremos un reporte con la librería ydata_profiling

En el ejercicio final utilizaremos la librería scrapy para extraer los datos de un catálogo de vinilos de segunda mano (https://www.revolverrecords.es/musica/segunda-mano/) y produciremos un dataset limpio con dicha información.

# 0. Importamos todas las librerías

In [1]:
import pandas as pd
import re

# BeautifulSoup
import requests
from bs4 import BeautifulSoup

#Selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

#creación reportes
from ydata_profiling import ProfileReport

#Scrapy
import json
import logging
from scrapy.crawler import CrawlerProcess
import scrapy


import warnings
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings('ignore')

/Users/juanviatela/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# 1. BeautifulSoup y Selenium con el sitio http://quotes.toscrape.com

## 1.1. BeautifulSoup

In [2]:
#creamos los headers de nuestro dataset
df_citas = pd.DataFrame(columns=['Texto', 'Autor', 'Etiquetas'])

In [3]:
#iteramos sobre el contenido de la página (y sobre sus primeras 10 páginas) para extraer el texto de las citas, 
#su autor y los tags. 

#Dentro del loop creamos el objeto soup para tener acceso y analizar el html de cada una de las páginas

for i in range(1,11):
    url = "https://quotes.toscrape.com/page/" + str(i) + "/"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    cites = soup.find_all("div", class_="quote")
    for cita in cites:
        text = cita.find("span", class_="text").text
        author = cita.find("small", class_="author").text
        tag_html = cita.find_all("a", class_="tag")
        tag_list=[]
        for tag in tag_html:
            tag_list.append(tag.text)
        df_citas = pd.concat([df_citas, pd.DataFrame([{'Texto': text, 'Autor':author, 'Etiquetas': tag_list}])], ignore_index=True)


In [4]:
#chequeamos las dimensiones del dataset y sus primeras líneas
print(df_citas.shape)
df_citas.head()

(100, 3)


,Texto,Autor,Etiquetas
0,“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”,Albert Einstein,"[change, deep-thoughts, thinking, world]"
1,"“It is our choices, Harry, that show what we truly are, far more than our abilities.”",J.K. Rowling,"[abilities, choices]"
2,“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”,Albert Einstein,"[inspirational, life, live, miracle, miracles]"
3,"“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”",Jane Austen,"[aliteracy, books, classic, humor]"
4,"“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”",Marilyn Monroe,"[be-yourself, inspirational]"


In [5]:
#Exportamos en formato csv
df_citas.to_csv("df_citas_bs.csv", index = False)

## 1.2 Selenium

In [6]:
#accedemos al navegador y establecemos las opciones de la librería
options = Options()
options.add_argument("start-maximized")
browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)

In [7]:
#creamos los headers de nuestro dataset
df_citas_sel = pd.DataFrame(columns=['Texto', 'Autor', 'Etiquetas'])

In [8]:
#iteramos sobre todas las páginas del sitio y en cada una de ellas volvemos a iterar para extraer la info
for i in range(1,11):
    url = "https://quotes.toscrape.com/page/" + str(i) + "/"
    browser.get(url)
    cites = browser.find_elements(By.CLASS_NAME, 'quote')
    for cita in cites:
        text=cita.find_element(By.CLASS_NAME, 'text').text
        author=cita.find_element(By.CLASS_NAME, 'author').text
        tag_html=cita.find_elements(By.CLASS_NAME, 'tags')
        tag_list=[]
        for tag in tag_html:
            tag_list.append(tag.text)
        df_citas_sel = pd.concat([df_citas_sel, pd.DataFrame([{'Texto': text, 'Autor':author, 'Etiquetas': tag_list}])], ignore_index=True)
browser.quit()

In [9]:
#chequeamos las dimensiones del dataset y sus primeras líneas
print(df_citas_sel.shape)
df_citas_sel.head()

(100, 3)


,Texto,Autor,Etiquetas
0,“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”,Albert Einstein,[Tags: change deep-thoughts thinking world]
1,"“It is our choices, Harry, that show what we truly are, far more than our abilities.”",J.K. Rowling,[Tags: abilities choices]
2,“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”,Albert Einstein,[Tags: inspirational life live miracle miracles]
3,"“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”",Jane Austen,[Tags: aliteracy books classic humor]
4,"“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”",Marilyn Monroe,[Tags: be-yourself inspirational]


In [10]:
#Exportamos en formato csv
df_citas_sel.to_csv("df_citas_sel.csv", index = False)

# 2.  BeautifulSoup y Selenium con el sitio https://en.wikipedia.org/wiki/2022%E2%80%9323_New_York_Knicks_season

## 2.1 BeautifulSoup

In [11]:
#accedemos a la página y creamos el objeto soup para analizar el html
URL = "https://en.wikipedia.org/wiki/2022%E2%80%9323_New_York_Knicks_season"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

In [12]:
#buscamos las tablas (que en wikipedia aparecen como class="wikitable")
tables = soup.find_all("table", class_="wikitable")

In [13]:
#Seleccionamos la tabla 12 que contiene los datos que nos interesan e imprimimos su contenido para entender su 
#estructura
statistics_table = tables[13]
print(statistics_table)

<table class="wikitable sortable plainrowheaders" style="text-align:right;">
<caption>New York Knicks statistics
</caption>
<tbody><tr>
<th scope="col" style="background-color: #1D428A; color: #FFFFFF; box-shadow: inset 2px 2px 0 #F58426, inset -2px -2px 0 #F58426;">Player
</th>
<th scope="col" style="background-color: #1D428A; color: #FFFFFF; box-shadow: inset 2px 2px 0 #F58426, inset -2px -2px 0 #F58426;">GP
</th>
<th scope="col" style="background-color: #1D428A; color: #FFFFFF; box-shadow: inset 2px 2px 0 #F58426, inset -2px -2px 0 #F58426;">GS
</th>
<th scope="col" style="background-color: #1D428A; color: #FFFFFF; box-shadow: inset 2px 2px 0 #F58426, inset -2px -2px 0 #F58426;">MPG
</th>
<th scope="col" style="background-color: #1D428A; color: #FFFFFF; box-shadow: inset 2px 2px 0 #F58426, inset -2px -2px 0 #F58426;">FG%
</th>
<th scope="col" style="background-color: #1D428A; color: #FFFFFF; box-shadow: inset 2px 2px 0 #F58426, inset -2px -2px 0 #F58426;">3FG%
</th>
<th scope="col" 

In [14]:
#de la tabla seleccionada extraemos los headers ('th') e iteramos para poblar una lista vacía
headers= statistics_table.findAll("th")
columns=[]
for header in headers:
    columns.append(header.text)
print(columns)

['Player\n', 'GP\n', 'GS\n', 'MPG\n', 'FG%\n', '3FG%\n', 'FT%\n', 'RPG\n', 'APG\n', 'SPG\n', 'BPG\n', 'PPG\n']


In [15]:
#reemplazamos los saltos de línea por espacios vacios 
columns = [elemento.replace('\n', '') for elemento in columns]
print(columns)

['Player', 'GP', 'GS', 'MPG', 'FG%', '3FG%', 'FT%', 'RPG', 'APG', 'SPG', 'BPG', 'PPG']


In [16]:
#creamos el dataset solo con las columnas
df_knicks_stats = pd.DataFrame(columns=columns)

In [17]:
#extramos la información de las filas ('tr')
rows = statistics_table.findAll("tr")

In [18]:
#descartamos la fila 0 que contiene los headers
rows = rows[1:]

In [19]:
#iteramos por cada fila para extraer la información de cada una de las columnas como texto
#concatenamos el dataframe con los headers y el diccionario con la info extraida
for row in rows:
    cells = row.findAll("td")
    player = cells[0].text
    gp = cells[1].text
    gs = cells[2].text
    mpg = cells[3].text
    fg = cells[4].text
    threefg =  cells[5].text
    ft = cells[6].text
    rpg = cells[7].text
    apg = cells[8].text
    spg = cells[9].text
    bpg = cells[10].text
    ppg = cells[11].text
    df_knicks_stats = pd.concat([df_knicks_stats, pd.DataFrame([{'Player': player, 
                                                                 'GP' : gp, 
                                                                 'GS' : gs, 
                                                                 'MPG' : mpg, 
                                                                 'FG%' : fg, 
                                                                 '3FG%' : threefg, 
                                                                 'FT%' : ft, 
                                                                 'RPG' : rpg, 
                                                                 'APG' : apg, 
                                                                 'SPG' : spg, 
                                                                 'BPG' : bpg, 
                                                                 'PPG' : ppg}])], ignore_index=True)

In [20]:
#echamos un vistazo al dataframe
df_knicks_stats.head(1)

,Player,GP,GS,MPG,FG%,3FG%,FT%,RPG,APG,SPG,BPG,PPG
0,Ryan Arcidiacono,11,0,2.4,.200,.333,—,.4,.2,.2,.0,.3\n


In [21]:
#eliminamos el salto de linea de la columa 'PPG'
df_knicks_stats['PPG'] = df_knicks_stats['PPG'].str.replace('\n', '')
df_knicks_stats

,Player,GP,GS,MPG,FG%,3FG%,FT%,RPG,APG,SPG,BPG,PPG
0,Ryan Arcidiacono,11,0,2.4,.200,.333,—,.4,.2,.2,.0,.3
1,RJ Barrett,73,73,33.9,.434,.310,.740,5.0,2.8,.4,.2,19.6
2,Jalen Brunson,68,68,35.0,.491,.416,.829,3.5,6.2,.9,.2,24.0
3,Evan Fournier,27,7,17.0,.337,.307,.857,1.8,1.3,.6,.1,6.1
4,Quentin Grimes,71,66,29.9,.468,.386,.796,3.2,2.1,.7,.4,11.3
5,Josh Hart,25,1,30.0,.586,.519,.789,7.0,3.6,1.4,.5,10.2
6,Isaiah Hartenstein,82,8,19.8,.535,.216,.676,6.5,1.2,.6,.8,5.0
7,Trevor Keels,3,0,2.7,.250,.250,—,.7,.0,.0,.0,1.0
8,Miles McBride,64,2,11.9,.358,.299,.667,.8,1.1,.6,.1,3.5
9,Sviatoslav Mykhailiuk,13,0,3.1,.500,.600,.600,.5,.1,.1,.0,1.6


In [22]:
#exportamos el dataset a un archivo csv
df_knicks_stats.to_csv('knicks_stats_2022-2023_bs.csv', index = False)

## 2.2 Selenium

In [23]:
#accedemos a través del navegador a la página web y establecemos las opciones de la librería

URL = "https://en.wikipedia.org/wiki/2022%E2%80%9323_New_York_Knicks_season"
options = Options()
options.add_argument("start-maximized")
browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)
browser.get(URL)

In [24]:
#a través del By.CLASS_NAME buscamos todas las tablas identificadas como 'wikitable'
tablas = browser.find_elements(By.CLASS_NAME, 'wikitable')

In [25]:
#identificamos la tabla que nos interesa y creamos un objeto con ella
tablas_knicks = tablas[13]

In [26]:
#extraemos los headers y creamos un dataframe
headers= tablas_knicks.find_elements(By.TAG_NAME, "th")
columns=[]
for header in headers:
    columns.append(header.text)

df_knicks_stats_sel = pd.DataFrame(columns = columns)
df_knicks_stats_sel

,Player,GP,GS,MPG,FG%,3FG%,FT%,RPG,APG,SPG,BPG,PPG


In [27]:
#extraemos la información de las filas comenzando desde la 1, pues la 0 tiene la info de los headers
rows = tablas_knicks.find_elements(By.TAG_NAME, "tr")
rows = rows[1:]

In [28]:
#iteramos sobre las filas para extaers la información una a una y la ingresamos en nuestro dataset
for row in rows:
    cells = row.find_elements(By.TAG_NAME, "td")
    player = cells[0].text
    gp = cells[1].text
    gs = cells[2].text
    mpg = cells[3].text
    fg = cells[4].text
    threefg =  cells[5].text
    ft = cells[6].text
    rpg = cells[7].text
    apg = cells[8].text
    spg = cells[9].text
    bpg = cells[10].text
    ppg = cells[11].text
    df_knicks_stats_sel = pd.concat([df_knicks_stats_sel, pd.DataFrame([{'Player': player, 
                                                                         'GP' : gp, 
                                                                         'GS' : gs, 
                                                                         'MPG' : mpg, 
                                                                         'FG%' : fg, 
                                                                         '3FG%' : threefg, 
                                                                         'FT%' : ft, 
                                                                         'RPG' : rpg, 
                                                                         'APG' : apg, 
                                                                         'SPG' : spg, 
                                                                         'BPG' : bpg, 
                                                                         'PPG' : ppg}])], ignore_index=True)

In [29]:
#revisamos el dataset
df_knicks_stats_sel

,Player,GP,GS,MPG,FG%,3FG%,FT%,RPG,APG,SPG,BPG,PPG
0,Ryan Arcidiacono,11,0,2.4,.200,.333,—,.4,.2,.2,.0,.3
1,RJ Barrett,73,73,33.9,.434,.310,.740,5.0,2.8,.4,.2,19.6
2,Jalen Brunson,68,68,35.0,.491,.416,.829,3.5,6.2,.9,.2,24.0
3,Evan Fournier,27,7,17.0,.337,.307,.857,1.8,1.3,.6,.1,6.1
4,Quentin Grimes,71,66,29.9,.468,.386,.796,3.2,2.1,.7,.4,11.3
5,Josh Hart,25,1,30.0,.586,.519,.789,7.0,3.6,1.4,.5,10.2
6,Isaiah Hartenstein,82,8,19.8,.535,.216,.676,6.5,1.2,.6,.8,5.0
7,Trevor Keels,3,0,2.7,.250,.250,—,.7,.0,.0,.0,1.0
8,Miles McBride,64,2,11.9,.358,.299,.667,.8,1.1,.6,.1,3.5
9,Sviatoslav Mykhailiuk,13,0,3.1,.500,.600,.600,.5,.1,.1,.0,1.6


In [30]:
#exportamos el dataframe a un .csv
df_knicks_stats_sel.to_csv('knicks_stats_2022-2023_sel.csv', index = False)

# 3. Crear un reporte del dataframe extraido

En esta sección utilizaremos el método ProfileReport de la librería ydata_profiling para crear un reporte sobre nuestro dataset, pero antes revisaremos los datatype de cada fila para que el informe sea correcto

In [31]:
df_knicks_stats_sel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Player  17 non-null     object
 1   GP      17 non-null     object
 2   GS      17 non-null     object
 3   MPG     17 non-null     object
 4   FG%     17 non-null     object
 5   3FG%    17 non-null     object
 6   FT%     17 non-null     object
 7   RPG     17 non-null     object
 8   APG     17 non-null     object
 9   SPG     17 non-null     object
 10  BPG     17 non-null     object
 11  PPG     17 non-null     object
dtypes: object(12)
memory usage: 1.7+ KB


Convertiremos todas las columnas object en int o float, menos la columna 'Player'

In [32]:
numeric_columns = df_knicks_stats_sel.columns.difference(['Player'])
df_knicks_stats_sel[numeric_columns] = df_knicks_stats_sel[numeric_columns].apply(pd.to_numeric, errors='coerce')
print(df_knicks_stats_sel.dtypes)

Player     object
GP          int64
GS          int64
MPG       float64
FG%       float64
3FG%      float64
FT%       float64
RPG       float64
APG       float64
SPG       float64
BPG       float64
PPG       float64
dtype: object


In [33]:
#construimos el informe definiendo su contenido
knicks_stats_profile = ProfileReport(df_knicks_stats_sel, 
                  title = 'Information on the players statistics of 2022-2023 New York Knicks Basketball Team', 
                  dataset = {'description' : 'This dataset contains information about the New York Knicks players statistics 2022-2023 season', 
                            'url': 'https://en.wikipedia.org/wiki/2022%E2%80%9323_New_York_Knicks_season'}, 
                  variables = {'descriptions': {
                      'Player' : 'Player Name',  
                      'GP' : 'Games Played', 
                      'GS' : 'Games Start', 
                      'MPG' : 'Minutes Per Game', 
                      'FG%' : 'Field Goal Percentage', 
                      '3FG%' : '3 Point Field Goal Percentage',
                      'FT%' : 'Free Trow Percentage',
                      'RPG' : 'Rebounds Per Game', 
                      'APG' : 'Assist Per Game',
                      'SPG' : 'Steals Per Game', 
                      'BPG' : 'Blocks Per Game',
                      'PPG' : 'Points Per Game',
                  }})
#exportamos el informe en una página html
knicks_stats_profile.to_file('Knicks_Player_Stats_2022-2023.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# 4. Scrapy del sitio https://www.revolverrecords.es/musica/segunda-mano/

In [34]:
#construimos un pipeline para iniciar y cerrar nuestra araña y generar un archivo json en el cual escribiremos 
#la información extraida como líneas

class JsonWriterPipeline(object):
    def open_spider(self, spider):
        self.file = open('vinyls_results.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        with open('vinyls_results.jl', 'a') as f:
            f.write(line)
        spider.logger.info(f"Item processed: {item}")
        return item
    
#En la clase VinylsSpider definimos: 
    # - url de la página de donde extaeremos la info
    # - configuración para realizar la extracción que incluye: 
        # -configuración de asumir que la extracción es hecha por un humano
        # -llamada al pipeline
        # -configuración del archivo json
    # - la identificación y el análisis de la info a extraer utilizando el método .css y yield
    # - un condicional para buscar en todas las páginas del cátalogo

class VinylsSpider(scrapy.Spider):
    name = "vinyls"
    start_urls = ['https://www.revolverrecords.es/musica/segunda-mano/']

    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
        'COOKIES_ENABLED': False,  # Deshabilitar cookies para simplificar
        'DOWNLOAD_DELAY': 2,  # Pausa de 2 segundos entre solicitudes
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1},
        'FEEDS': {'vinyls_results.json': {'format': 'json', 'overwrite': True}},
    }

    def parse(self, response):
        for vinyls in response.css('ul.products.columns-6 li'):
            yield { 'Title': vinyls.css('h2.woocommerce-loop-product__title::text').extract_first(),
                    'Price': vinyls.css('span.price > span.woocommerce-Price-amount > bdi::text').extract_first(),
                    'Format': vinyls.css('div.product-attributes > span.pa_formato::text').extract_first()
                                      }
            
        next_page_relative = response.css('a.page-numbers::attr(href)').extract_first()
        if next_page_relative:
            next_page = response.urljoin(next_page_relative)
            yield scrapy.Request(url=next_page, callback=self.parse)
            
#procesamos la araña utilizando la clase creada anteriormente y generamos el archivo json
process = CrawlerProcess({
    'LOG_LEVEL': 'WARNING',
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    'FEEDS': {'vinyls_results.json': {'format': 'json', 'overwrite': True}},
})

process.crawl(VinylsSpider)
process.start()
process.stop()

<DeferredList at 0x13bdd5290 current result: []>

In [36]:
#importamos el json como un dataframe y revisamos sus dimensiones y primeras líneas
df_vinyls = pd.read_json('vinyls_results.jl', lines=True)
print(df_vinyls.shape)
df_vinyls.head(10)

(67, 3)


,Title,Price,Format
0,OBITUARY – SLOWLY WE ROT 2A MA NM.NM.,80,LP
1,EXODUS – IMPACT IS IMMINENT 2A MA NM.NM.,70,LP
2,SEPULTURA – ARISE 2A MA NM.NM.,100,LP
3,METALLICA – …AND JUSTICE FOR ALL 2A MA NM.NM.,60,2LP
4,MOTORHEAD – 1916 2A MA NM.NM.,60,CD.
5,EVO – DURACION DE LO ETERNO 2A MA NM.NM.,200,LP
6,OVERDOSE/SEPULTURA – SECULO XX/BESTIAL DEVASTATION 2A MA NM.VG+,250,LP
7,DESTRUCTION – RELEASE FROM AGONY 2A MA NM.NM.,35,LP
8,BAD RELIGION – GENERATOR 2A MA NM.NM.,50,CD.
9,EXCITER – LONG LIVE THE LOUD 2A MA NM.VG+.,30,LP


Antes de extraer el archivo tenemos que comentar que por la estructura del html de la página de donde extrajimos la información la columna 'Title' incluye el nombre del artista, el título del disco y la condición del mismo. Para terminar de limpiar nuestro dataset deberíamos utilizar regex pero eso está más allá del objetivo de este proyecto. 

In [102]:
df_vinyls.to_csv('df_vinyls.csv', index = False)